In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'zomato-bangalore-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F926506%2F1567895%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240625%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240625T160109Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D964f4678a2761d05991ca81f0f5e61f813187550ea41bc7861beaaff283ef1d77868303ec8e83643da413b3df88afabd78667d15966648f0c1f0fa0f93604342291755b87eb4c7f616231b2eb29001a487024e179e7b70856dd61a5c79d002eb3b90f77f7f4d129763aa7e4e5b117bd23eb23a0c2f758345b4dd936c449e3412c7ef1eb77f7a08ad60db04b8ef6fdaebc39bdbf86f50dd6805eec41c78b18efceabf5628ef571c2d7b2b69392aeb52919178eda2f805ec9b2abe6cf2eaeba48bc9597982b02b8a1fffd482e1b1d726027b5ce36248c91c3a07b24da93e350f9bd5210b39eac42aca8ee731daa09c0744601d8a10aebd8e231a234ed0e8ed8b88'
DATA_SOURCE_MAPPING = os.path.expanduser(DATA_SOURCE_MAPPING),'UTF=8'
KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


AttributeError: 'tuple' object has no attribute 'split'

## How the Restaurant Recommendation System Works?

One of the type of programming in ML is data filtering and data cleaning which is technically used with syntaxes in python, some of the types related to recommendation system are reataurant recommendation system. Recommendation systems are defined as a technique which used to recommend or regenerate something. For example: The methods used for recommend a cuisine of a resraurant.
It is of two types-


1. Content-based filtering
2. Collaborative filtering
https://www.bing.com/ck/a?!&&p=14d006f2a075f6ddJmltdHM9MTcxOTQ0NjQwMCZpZ3VpZD0yMzllNmViMC1lNDljLTYzMDMtMDcyNC03YTNiZTUzNDYyZWYmaW5zaWQ9NTUyMA&ptn=3&ver=2&hsh=3&fclid=239e6eb0-e49c-6303-0724-7a3be53462ef&psq=Recommendation+system+types+&u=a1aHR0cHM6Ly9jZW9iZWUuZGV2L2Jsb2cvdW5kZXJzdGFuZGluZy1yZWNvbW1lbmRhdGlvbi1zeXN0ZW1zLXR5cGVzLWFwcGxpY2F0aW9ucy1hbmQtY2hhbGxlbmdlcyM6fjp0ZXh0PVR5cGVzJTIwb2YlMjBSZWNvbW1lbmRhdGlvbiUyMFN5c3RlbXMlMjAxJTIwQ29sbGFib3JhdGl2ZSUyMEZpbHRlcmluZyUzQSUyMFRoaXMsRmlsdGVyaW5nJTIwdG8lMjBwcm92aWRlJTIwbW9yZSUyMGFjY3VyYXRlJTIwYW5kJTIwZGl2ZXJzZSUyMHJlY29tbWVuZGF0aW9ucy4&ntb=1

I will start the task of Restaurant Recommendation System by importing the necessary Python Libraries:

In [2]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

ModuleNotFoundError: No module named 'numpy'

Now, I will load and read the dataset:

In [ ]:
zomato_app=pd.read_csv("/content/Dataset .csv")
zomato_app.head() # prints the first 5 rows of the dataset
dataset = zomato_app.copy()

Now the next step is data cleaning and feature engineering for this step we need to do a lot of stuff with the data such as:

1. Deleting Unnecessary Columns
2. Removing the Duplicates
3. Remove the NaN values from the dataset
4. Changing the column names
5. Data Transformations
6. Data Cleaning
7. Adjust the column names
Now, let’s perform all the above steps in our data:

In [ ]:
#Deleting Unnnecessary Columns
df = pd.dataframe(zomato_app)
zomato_app = zomato_app.drop(['url','dish_liked', 'phone'],axis=1): #Dropping the column "dish_liked", "phone", "url" and saving the new dataset as "zomato"

#Removing the Duplicates
zomato_app.duplicated().sum()
zomato_app.drop_duplicates(inplace=True)

#Remove the NaN values from the dataset
zomato_app.isnull().sum()
zomato_app.dropna(how='any',inplace=True)

#Changing the column names
zomato_app = zomato_app.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type', 'listed_in(city)':'city'})

#Some Transformations
zomato['cost'] = zomato['cost'].astype(str) #Changing the cost to string
zomato['cost'] = zomato['cost'].apply(lambda x: x.replace(',','.')) #Using lambda function to replace ',' from cost
zomato['cost'] = zomato['cost'].astype(float)
#Removing '/5' from Rates
zomato_app = zomato_app.loc[zomato.rate !='NEW']
zomato_app = zomato_app.loc[zomato.rate !='-'].reset_index(drop=True)
remove_slash = lambda x: x.replace('/5', '') if type(x) == np.str else x
zomato_app.rate = zomato_app.rate.apply(remove_slash).str.strip().astype('float')

# Adjust the column names
zomato_app.name = zomato.name.apply(lambda x:x.title())
zomato_app.online_order.replace(('Yes','No'),(True, False),inplace=True)
zomato_app.book_table.replace(('Yes','No'),(True, False),inplace=True)

## Computing Mean Rating
restaurants = list(zomato['name'].unique())
zomato_app['Mean Rating'] = 0

for i in range(len(restaurants)):
    zomato['Mean Rating'][zomato['name'] == restaurants[i]] = zomato['rate'][zomato['name'] == restaurants[i]].mean()

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (1,5))
zomato[['Mean Rating']] = scaler.fit_transform(zomato[['Mean Rating']]).round(2)

SyntaxError: invalid syntax (<ipython-input-181-964714d2b6bc>, line 3)

Now the next step is to perform some text preprocessing steps which include:

1. Lower casing
2. Removal of Punctuations
3. Removal of Stopwords
4. Removal of URLs
5. Spelling correction

Now let’s perform the above text preprocessing steps on the data:

In [ ]:
## Lower Casing
zomato == zomato_app.align[[("reviews_list")] = zomato_app[("reviews_list")].str.lower()]

## Removal of Punctuations
zomato_app.head()

## Removal of Puctuations
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_punctuation(text))

## Removal of Stopwords
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_stopwords(text))

## Removal of URLS
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_urls(text))

zomato[['reviews_list', 'cuisines']].sample(5)

SyntaxError: invalid syntax (<ipython-input-191-6c821661bfd2>, line 2)

In [ ]:

#RESTAURANT NAMES
restaurant_names = list(zomato['name'].unique())
def get_top_words(column, top_nu_of_words, nu_of_word):
    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')
    bag_of_words = vec.fit_transform(column)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:top_nu_of_words]

zomato=zomato_app.drop(['address','rest_type', 'type', 'menu_item'],axis=1)
import pandas

# Randomly sample 60% of your dataframe
df_percent = zomato.sample(frac=0.5)

## TF-IDF Vectorization

TF-IDF (Term Frequency-Inverse Document Frequency) used for determining statiscal document vectorization. This will give you a matrix where each column represents a word in the general vocabulary (all words that appear in at least one document) and each column represents a restaurant, as before.

TF-IDF is the statistical method used for assessing the meaning of a word in a given document. Now, I will use the TF-IDF vectorization on the dataset:

In [196]:
df_percent.set_index('Value', inplace=True):
df_percent.head()
feature = data ["Type"].tolist()
text.TfidfVectorizer(input=feature, stop_words = "english")
tfidf_matrix = text.fit_transform(feature)
similarity_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(df_percent.index)

# Creating tf-idf matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

SyntaxError: invalid syntax (<ipython-input-196-f48355de095f>, line 1)

Now the last step for creating a Restaurant Recommendation System is to write a function that will recommend restaurants:

In [ ]:
from types import new_class
     def recommend(name, cosine_similarities = cosine_similarities), (indices = indices), (df_percent = df_percent.html_escape()):

    # Create a list to put top restaurants
    recommend_restaurant = []

    # Find the index of the hotel entered
    idx = indices[indices == name].index[0]

    # Find the restaurants with a similar cosine-sim value and order them from bigges number
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)

    # Extract top 30 restaurant indexes with a similar cosine-sim value
    top30_indexes = list(score_series.iloc[0:31].index)

    # Names of the top 30 restaurants
    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])

    # Creating the new data set to show similar restaurants
    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])

    # Create the top 30 similar restaurants with some of their columns
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df_percent[['cuisines','Mean Rating', 'cost']][df_percent.index == each].sample()))

    # Drop the same named restaurants and sort only the top 10 by the highest rating
    df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)

    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))

    return df_new:
      recommend('Pai Vihar')

for each in top45_dishes:
    print(each)
    if len(df_new_dish) < 100:
        df_new = df_new.append(pd.DataFrame(df_percent[['cuisines','dishes']][df.value_index == each].sample()))
    else:
        print('No similar dishes found')

    elif len(df_new_dish) == 100:
        break:

    df_new_dish = df_new_dish.delete_duplicates(subset=['cuisines','dishes', 'menu'], keep=False)
    df_new_dish = df_new_dish.drop_values(by='dishes', ascending=False).head(10)
    print(top_k_score_menu, 'menu hooked')

  #df_new_class.insert('zomato') and assert("recommendation system"):

import sys

df_new, Recommendation_System_Zomato(n): # Return factorial
    output = 1:

    for i in range (1,n):
        result = result * i
    print [("Recommendation_System_Zomato"),result]

     return output:

    print( "Recommendation system Zomato")




IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)

#### As as you can see that we got a fairly good output. So, I hope you liked this article on Machine Learning project on Restaurant Recommendation system with Python programming language. Feel free to ask your valuable questions in the comments section